In [4]:
import gzip
import csv

phedex_columns= ['now_sec','dataset_name','dataset_id','dataset_is_open','create_time',
                'update_time','block_name','block_id','block_files','block_bytes',
                'block_is_open','block_time_create','block_time_update',
                'node_name','node_id','br_is_active','br_src_files',
                'br_src_bytes','br_dest_files','br_dest_bytes',
                'br_node_files','br_node_btyes','br_xfer_files',
                'br_xfer_bytes','br_is_custodial','br_user_group_id',
                'replica_time_create','replica_time_updater']
phedex_colmap={}
for i in range(len(phedex_columns)):
    phedex_colmap[phedex_columns[i]]=i

dataset_col=phedex_colmap['dataset_name']
site_col=phedex_colmap['node_name']
block_col=phedex_colmap['block_name']
replicaTime_col=phedex_colmap['replica_time_create']
size_col=phedex_colmap['block_bytes']
group_col=phedex_colmap['br_user_group_id']
phedex_info={}
n=0
fmt='%Y-%m-%d'
import datetime
import numpy
sumBytes=0
datasets={}
dataset_sizes={}
dataset_total={}
with gzip.open('phedex.181024.gz') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        siteName=row[site_col]
        dataset_name=row[dataset_col]
        #duplicate AOD is ok (maybe not)
        if 'AOD' in row[dataset_col]: continue
        # get rid of tape and obsolete stuff
        if 'MSS' in siteName: continue
        if 'Export' in siteName: continue
        if 'Buffer' in siteName: continue
        if 'T3' in siteName: continue
        if 'Stage' in siteName: continue
        if 'XT1' in siteName: continue
        if 'XT2' in siteName: continue
        if row[group_col]!='42': continue
        
        if row[dataset_col] not in datasets:
            datasets[dataset_name]={}
            dataset_sizes[dataset_name]={}
            dataset_total[dataset_name]=0
        blocks=datasets[dataset_name]
        block_sizes=dataset_sizes[dataset_name]
        bltmp=row[block_col].split('#')[1]
        if bltmp not in blocks:
            blocks[bltmp]=[]
            block_sizes[bltmp]=row[size_col]
            dataset_total[dataset_name]+=float(row[size_col])
        blocks[bltmp].append(siteName)
        #print row[block_col]             
        if n==0:
            for i in range(len(phedex_columns)):
                print i,row[i],phedex_columns[i]
        #if n==300: break
        n=n+1
print len(datasets)

0 1540346322.000000000000000000000000000004 now_sec
1 /SingleMu/Run2012B-v1/RAW dataset_name
2 533368 dataset_id
3 y dataset_is_open
4 1336591558.22044 create_time
5 null update_time
6 /SingleMu/Run2012B-v1/RAW#ff79a718-9a85-11e1-a7a2-003048caaace block_name
7 3457872 block_id
8 1 block_files
9 2310840655 block_bytes
10 n block_is_open
11 1336643680.22186 block_time_create
12 1336644552.02913 block_time_update
13 T2_RU_INR node_name
14 561 node_id
15 n br_is_active
16 0 br_src_files
17 0 br_src_bytes
18 1 br_dest_files
19 2310840655 br_dest_bytes
20 1 br_node_files
21 2310840655 br_node_btyes
22 0 br_xfer_files
23 0 br_xfer_bytes
24 n br_is_custodial
25 42 br_user_group_id
26 1456517771 replica_time_create
27 1522732525.4199 replica_time_updater
4241


In [2]:
sorted_datasets = sorted(dataset_total.items(), key=lambda x: x[1])
sorted_datasets.reverse()


In [3]:
print len(datasets)
print ''
deleteable_blocks={}
deleteable_bytes={}
total_blocks={}
total_bytes={}
site_list={}
complex={}
for dataset,dummy in sorted_datasets:
    blocks= datasets[dataset]
    sites=[]
    complex_dataset=False
    num_blocks=len(blocks)
    ds_size=0
    
    for i,block in enumerate(blocks):
        ds_size+=float(dataset_sizes[dataset][block])
        if i==0: 
            nSites=len(blocks[block])
            sites_exp=sorted(blocks[block])
        if len(blocks[block])==1: continue #only at one site
        if dataset not in deleteable_blocks:
            deleteable_blocks[dataset]=0
            deleteable_bytes[dataset]=0
            
        deleteable_blocks[dataset]+=len(blocks[block])-1
        deleteable_bytes[dataset]+=(len(blocks[block])-1)*float(dataset_sizes[dataset][block])
        if nSites != len(blocks[block]):
            complex_dataset=True
        else:
            if sorted(blocks[block]) != sites_exp:
                complex_dataset=True
        for site in blocks[block]:
            if site not in sites:
                sites.append(site)
    complex[dataset]=complex_dataset
    total_blocks[dataset]=num_blocks
    total_bytes[dataset]=ds_size
    site_list[dataset]=sites
    
cand_ds=sorted(deleteable_bytes.items(), key= lambda x: x[1])
cand_ds.reverse()
for dataset,dummy in cand_ds:    
    tb_size=deleteable_bytes[dataset]/1024./1024./1024./1024.
    ds_size=total_bytes[dataset]
    sites=site_list[dataset]
    num_blocks=total_blocks[dataset]
    if tb_size>1:
        print 'Can remove',deleteable_blocks[dataset],'blocks with',int(tb_size),'TB from',dataset,
        print 'which has',num_blocks,'blocks and',int(ds_size/1024./1024./1024./1024.),'TB'
        if not complex_dataset:
            print 'Sites with all data',sites
        else:
            print 'Sites with some of the data',sites
        print ''
            

81610

Can remove 3395 blocks with 1603 TB from /SingleMuon/Run2018D-ZMu-PromptReco-v2/RAW-RECO which has 213 blocks and 100 TB
Sites with all data ['T2_US_Wisconsin', 'T2_ES_IFCA', 'T2_IT_Pisa', 'T2_UK_London_Brunel', 'T2_IT_Legnaro', 'T2_UK_SGrid_RALPP', 'T2_RU_JINR', 'T2_US_MIT', 'T2_AT_Vienna', 'T2_US_Caltech', 'T1_US_FNAL_Disk', 'T2_BE_IIHE', 'T2_RU_IHEP', 'T2_CH_CERN', 'T2_BR_SPRACE', 'T2_DE_DESY', 'T2_US_Purdue', 'T2_KR_KISTI']

Can remove 3110 blocks with 1274 TB from /MinBias_TuneCUETP8M1_14TeV-v1-pythia8/PhaseIITDRFall17GS-93X_upgrade2023_realistic_v2-v1/GEN-SIM which has 119 blocks and 48 TB
Sites with all data ['T2_UK_London_IC', 'T2_IT_Pisa', 'T2_ES_CIEMAT', 'T1_ES_PIC_Disk', 'T1_US_FNAL_Disk', 'T2_PL_Swierk', 'T1_UK_RAL_Disk', 'T2_IT_Legnaro', 'T2_DE_DESY', 'T2_US_Wisconsin', 'T1_FR_CCIN2P3_Disk', 'T2_US_Caltech', 'T2_DE_RWTH', 'T2_US_Purdue', 'T2_US_UCSD', 'T2_US_MIT', 'T2_US_Nebraska', 'T2_BE_IIHE', 'T2_UK_London_Brunel', 'T1_RU_JINR_Disk', 'T2_IT_Bari', 'T2_FR_IPHC', '

Can remove 417 blocks with 139 TB from /MET/Run2017F-EXOMONOPOLE-09May2018-v1/USER which has 231 blocks and 78 TB
Sites with all data ['T1_IT_CNAF_Disk', 'T1_RU_JINR_Disk', 'T1_UK_RAL_Disk', 'T2_RU_JINR', 'T2_CH_CERN', 'T2_US_Florida', 'T1_DE_KIT_Disk', 'T1_FR_CCIN2P3_Disk', 'T2_US_Purdue', 'T2_US_Caltech', 'T1_ES_PIC_Disk', 'T2_US_Vanderbilt', 'T2_US_Nebraska', 'T1_US_FNAL_Disk', 'T2_US_UCSD', 'T2_US_Wisconsin', 'T2_ES_CIEMAT']

Can remove 116 blocks with 129 TB from /MET/Run2018D-v1/RAW which has 98 blocks and 109 TB
Sites with all data ['T2_GR_Ioannina', 'T1_DE_KIT_Disk', 'T2_CH_CERN', 'T1_US_FNAL_Disk']

Can remove 164 blocks with 125 TB from /ZeroBias/Run2018A-v1/RAW which has 82 blocks and 60 TB
Sites with all data ['T2_UK_London_IC', 'T1_RU_JINR_Disk', 'T2_DE_RWTH', 'T2_US_Nebraska', 'T2_US_Caltech', 'T1_US_FNAL_Disk', 'T2_CH_CSCS']

Can remove 94 blocks with 118 TB from /Neutrino_E-10_gun/RunIISummer17PrePremix-101X_upgrade2018_realistic_v7-v1/GEN-SIM-DIGI-RAW which has 47 bloc

Can remove 207 blocks with 34 TB from /QCD_Pt_50to80_TuneCP5_13TeV_pythia8/RunIIFall17GS-93X_mc2017_realistic_v3-v1/GEN-SIM which has 68 blocks and 11 TB
Sites with all data ['T1_RU_JINR_Disk', 'T1_ES_PIC_Disk', 'T1_US_FNAL_Disk', 'T2_CH_CERN', 'T2_IT_Rome']

Can remove 469 blocks with 34 TB from /BsToJpsiPhi_BMuonFilter_SoftQCDnonD_TuneCP5_13TeV-pythia8-evtgen/RunIIFall17GS-93X_mc2017_realistic_v3-v1/GEN-SIM which has 484 blocks and 34 TB
Sites with all data ['T1_FR_CCIN2P3_Disk', 'T2_DE_DESY', 'T2_FR_GRIF_LLR', 'T2_US_Wisconsin', 'T1_RU_JINR_Disk', 'T1_IT_CNAF_Disk', 'T2_BE_IIHE', 'T2_IT_Pisa', 'T2_IT_Bari', 'T2_US_Purdue', 'T2_UK_London_IC', 'T2_PL_Swierk', 'T2_FR_GRIF_IRFU', 'T2_ES_CIEMAT', 'T2_IT_Legnaro', 'T2_US_UCSD', 'T1_ES_PIC_Disk', 'T2_US_MIT', 'T2_FR_CCIN2P3', 'T2_DE_RWTH', 'T2_US_Nebraska', 'T2_UK_London_Brunel', 'T2_FR_IPHC', 'T2_RU_JINR', 'T2_US_Caltech', 'T2_UK_SGrid_RALPP', 'T1_US_FNAL_Disk', 'T1_UK_RAL_Disk']

Can remove 170 blocks with 34 TB from /SinglePhoton/Run201

Can remove 38 blocks with 27 TB from /MET/Run2016B-v2/RAW which has 32 blocks and 20 TB
Sites with all data ['T2_US_Nebraska', 'T2_CH_CSCS', 'T2_US_UCSD', 'T2_CH_CERN']

Can remove 66 blocks with 27 TB from /WToLNu_1J_14TeV-madgraphMLM-pythia8/PhaseIITDRFall17DR-PU200_93X_upgrade2023_realistic_v2_ext1-v3/GEN-SIM-RECO which has 66 blocks and 27 TB
Sites with all data ['T1_UK_RAL_Disk', 'T2_DE_DESY']

Can remove 1729 blocks with 26 TB from /EGamma/Run2018D-EcalUncalZElectron-PromptReco-v2/ALCARECO which has 248 blocks and 3 TB
Sites with all data ['T2_UK_London_Brunel', 'T2_TW_NCHC', 'T2_RU_ITEP', 'T2_UK_SGrid_RALPP', 'T1_RU_JINR_Disk', 'T2_US_Florida', 'T2_RU_IHEP', 'T1_IT_CNAF_Disk', 'T2_CH_CERN']

Can remove 313 blocks with 25 TB from /ZeroBias/Run2018A-LumiPixelsMinBias-17Sep2018-v1/ALCARECO which has 108 blocks and 9 TB
Sites with all data ['T1_IT_CNAF_Disk', 'T2_CH_CERN', 'T2_RU_JINR', 'T1_RU_JINR_Disk', 'T1_DE_KIT_Disk', 'T2_UK_SGrid_RALPP', 'T2_UK_London_Brunel', 'T2_UK_London_IC

Can remove 31 blocks with 20 TB from /MinBias_TuneZ2star_8TeV-pythia6/Summer12-START50_V13-v3/GEN-SIM which has 14 blocks and 9 TB
Sites with all data ['T1_FR_CCIN2P3_Disk', 'T1_IT_CNAF_Disk', 'T2_CN_Beijing', 'T1_US_FNAL_Disk']

Can remove 16 blocks with 20 TB from /ZeroBias3/Run2018B-v1/RAW which has 16 blocks and 20 TB
Sites with all data ['T2_KR_KISTI', 'T2_US_Florida']

Can remove 49 blocks with 19 TB from /WToLNu_0J_14TeV-madgraphMLM-pythia8/PhaseIITDRFall17DR-PU200_93X_upgrade2023_realistic_v2_ext1-v1/GEN-SIM-RECO which has 49 blocks and 19 TB
Sites with all data ['T1_US_FNAL_Disk', 'T2_ES_CIEMAT']

Can remove 23 blocks with 19 TB from /MuonEG/Run2017F-v1/RAW which has 24 blocks and 19 TB
Sites with all data ['T2_DE_DESY', 'T2_UK_London_IC', 'T2_CH_CSCS', 'T1_RU_JINR_Disk', 'T2_FR_IPHC', 'T1_US_FNAL_Disk']

Can remove 27 blocks with 19 TB from /QCD_Pt_300to470_TuneCP5_13TeV_pythia8/RunIISummer18GS-101X_upgrade2018_realistic_v7-v1/GEN-SIM which has 38 blocks and 24 TB
Sites with 

...

Can remove 158 blocks with 2 TB from /EGamma/Run2018C-EcalUncalWElectron-17Sep2018-v1/ALCARECO which has 173 blocks and 3 TB
Sites with all data ['T2_US_UCSD', 'T1_IT_CNAF_Disk', 'T2_CH_CSCS', 'T2_US_Wisconsin', 'T2_US_Nebraska', 'T2_US_Purdue', 'T1_RU_JINR_Disk', 'T2_US_MIT', 'T2_DE_DESY', 'T2_UK_SGrid_RALPP', 'T2_UK_London_IC', 'T2_IN_TIFR', 'T1_UK_RAL_Disk', 'T2_US_Vanderbilt', 'T1_US_FNAL_Disk', 'T2_US_Caltech', 'T2_RU_JINR', 'T2_US_Florida', 'T2_CH_CERN']

Can remove 29 blocks with 2 TB from /MET/Run2018D-HighMET-PromptReco-v2/RAW-RECO which has 214 blocks and 25 TB
Sites with all data ['T1_DE_KIT_Disk', 'T2_CH_CERN']

Can remove 38 blocks with 2 TB from /DYToLL-M-50_3J_14TeV-madgraphMLM-pythia8/PhaseIITDRFall17DR-noPU_93X_upgrade2023_realistic_v2-v1/GEN-SIM-RECO which has 19 blocks and 1 TB
Sites with all data ['T2_BE_IIHE', 'T2_UK_London_IC', 'T1_US_FNAL_Disk']

Can remove 10 blocks with 2 TB from /GluGluHToZZTo4L_M125_14TeV_powheg2_JHUgenV702_pythia8/PhaseIITDRFall17DR-PU200_93

Can remove 23 blocks with 1 TB from /VBF_RS_bulk_M_2500_W10pc_14TeV-madgraph/PhaseIITDRFall17DR-PU200_93X_upgrade2023_realistic_v2-v1/GEN-SIM-RECO which has 24 blocks and 2 TB
Sites with all data ['T1_US_FNAL_Disk', 'T2_UK_London_IC', 'T2_IT_Bari', 'T2_CH_CERN', 'T1_UK_RAL_Disk', 'T1_RU_JINR_Disk', 'T2_BE_IIHE', 'T2_FR_GRIF_LLR', 'T2_UK_London_Brunel', 'T2_FR_GRIF_IRFU', 'T2_ES_CIEMAT', 'T1_DE_KIT_Disk', 'T2_DE_RWTH']

Can remove 18 blocks with 1 TB from /Pythia8_AllQCDPhoton120Flt50_Hydjet_Cymbal_MB/HiFall15-75X_mcRun2_HeavyIon_v14_ext1-v1/GEN-SIM which has 18 blocks and 1 TB
Sites with all data ['T2_DE_DESY', 'T2_US_Purdue']

Can remove 44 blocks with 1 TB from /ZeroBias/Run2018B-TkAlMinBias-17Sep2018-v1/ALCARECO which has 47 blocks and 3 TB
Sites with all data ['T1_FR_CCIN2P3_Disk', 'T1_IT_CNAF_Disk', 'T2_US_Nebraska', 'T2_CH_CSCS', 'T2_US_Caltech', 'T2_FR_GRIF_IRFU', 'T1_US_FNAL_Disk', 'T2_US_UCSD', 'T2_US_MIT', 'T2_CH_CERN', 'T2_BE_IIHE', 'T2_US_Vanderbilt', 'T2_ES_CIEMAT', 'T2_US

Can remove 61 blocks with 1 TB from /DoubleMuonLowMass/Run2017E-BPHSkim-PromptReco-v1/USER which has 61 blocks and 1 TB
Sites with all data ['T1_US_FNAL_Disk', 'T2_BR_SPRACE']

Can remove 1 blocks with 1 TB from /Charmonium/Run2018C-v1/RAW which has 27 blocks and 32 TB
Sites with all data ['T1_US_FNAL_Disk', 'T1_RU_JINR_Disk']

Can remove 2 blocks with 1 TB from /QCD_Pt-15to1000_MuEnrichedPt5_TuneCP5_13TeV_pythia8/RunIIFall18DR-NZSFlatPU28to62_102X_upgrade2018_realistic_v11-v1/GEN-SIM-RAW which has 3 blocks and 1 TB
Sites with all data ['T1_ES_PIC_Disk', 'T2_US_Purdue', 'T2_US_MIT']

Can remove 30 blocks with 1 TB from /MET/Run2018D-LogError-PromptReco-v2/RAW-RECO which has 215 blocks and 13 TB
Sites with all data ['T1_DE_KIT_Disk', 'T2_CH_CERN']

Can remove 5 blocks with 1 TB from /SUSYGluGluToBBHToBB_NarrowWidth_M-1200_TuneCP5_13TeV-pythia8/RunIIFall18DR-FlatPU28to62NZS_102X_upgrade2018_realistic_v11-v1/GEN-SIM-RAW which has 8 blocks and 2 TB
Sites with all data ['T1_ES_PIC_Disk', 'T

Can remove 2 blocks with 1 TB from /HIMinimumBias2/XeXeRun2017-v1/RAW which has 2 blocks and 1 TB
Sites with all data ['T2_CN_Beijing', 'T2_US_MIT']

Can remove 4 blocks with 1 TB from /SUSYGluGluToBBHToBB_NarrowWidth_M-350_TuneCP5_13TeV-pythia8/RunIIFall18GS-102X_upgrade2018_realistic_v11-v1/GEN-SIM which has 6 blocks and 1 TB
Sites with all data ['T1_ES_PIC_Disk', 'T2_US_Florida', 'T2_US_Purdue']

Can remove 201 blocks with 1 TB from /SingleMuon/Run2018C-DtCalib-17Sep2018-v1/ALCARECO which has 106 blocks and 0 TB
Sites with all data ['T2_CH_CSCS', 'T1_US_FNAL_Disk', 'T2_US_Caltech', 'T2_IT_Bari', 'T2_US_UCSD', 'T2_US_Vanderbilt', 'T2_IT_Legnaro', 'T2_US_Nebraska', 'T2_US_Florida', 'T2_US_Wisconsin', 'T2_US_Purdue']

Can remove 159 blocks with 1 TB from /QCD_HT1000to1500_TuneCP5_13TeV-madgraphMLM-pythia8/RunIISummer18wmLHEGS-101X_upgrade2018_realistic_v7-v1/LHE which has 164 blocks and 1 TB
Sites with all data ['T1_US_FNAL_Disk', 'T2_UK_London_IC', 'T2_FR_CCIN2P3', 'T2_US_Nebraska', '

Can remove 20 blocks with 1 TB from /NonPrDs-f0Pi_pThat-0_DsPT-2_HydjetCymbMB_5p02_Pythia8/HiFall15-75X_mcRun2_HeavyIon_v14-v1/GEN-SIM which has 26 blocks and 1 TB
Sites with all data ['T2_US_MIT', 'T1_ES_PIC_Disk', 'T1_DE_KIT_Disk']

